In [1]:
!pip install -q /kaggle/input/kerasv3-lib-ds/tensorflow-2.15.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --no-deps
!pip install -q /kaggle/input/kerasv3-lib-ds/keras-3.0.4-py3-none-any.whl --no-deps

In [2]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # # you can also use tensorflow or torch

import keras
import keras_nlp
from keras import ops
import tensorflow as tf

import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

import plotly.graph_objs as go
import plotly.express as px

2024-05-01 17:25:01.613741: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-01 17:25:01.613798: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-01 17:25:01.615197: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
print("KerasNLP:", keras_nlp.__version__)

TensorFlow: 2.15.0
Keras: 3.0.4
KerasNLP: 0.9.3


In [4]:
class CFG:
    seed = 42
    preset = "deberta_v3_small_en"  # name of pretrained backbone
    train_seq_len = 1024  # max size of input sequence for training
    train_batch_size = 2 * 8  # size of the input batch in training, x 2 as two GPUs
    infer_seq_len = 2000  # max size of input sequence for inference
    infer_batch_size = 2 * 2  # size of the input batch in inference, x 2 as two GPUs
    epochs = 6  # number of epochs to train
    lr_mode = "exp"  # lr scheduler mode from one of "cos", "step", "exp"

    labels = [
        "B-EMAIL",
        "B-ID_NUM",
        "B-NAME_STUDENT",
        "B-PHONE_NUM",
        "B-STREET_ADDRESS",
        "B-URL_PERSONAL",
        "B-USERNAME",
        "I-ID_NUM",
        "I-NAME_STUDENT",
        "I-PHONE_NUM",
        "I-STREET_ADDRESS",
        "I-URL_PERSONAL",
        "O",
    ]
    id2label = dict(enumerate(labels))  # integer label to BIO format label mapping
    label2id = {
        v: k for k, v in id2label.items()
    }  # BIO format label to integer label mapping
    num_labels = len(labels)  # number of PII (NER) tags

    train = True  # whether to train or use already trained ckpt

In [5]:
keras.utils.set_random_seed(CFG.seed)

In [6]:
# Get devices default "gpu" or "tpu"
devices = keras.distribution.list_devices()
print("Device:", devices)

if len(devices) > 1:
    # Data parallelism
    data_parallel = keras.distribution.DataParallel(devices=devices)

    # Set the global distribution.
    keras.distribution.set_distribution(data_parallel)

Device: ['gpu:0', 'gpu:1']


In [7]:
keras.mixed_precision.set_global_policy("mixed_float16")

In [8]:
BASE_PATH = "/kaggle/input/pii-detection-removal-from-educational-data"

In [9]:
# Train-Valid data
data = json.load(open(f"{BASE_PATH}/train.json"))

# Initialize empty arrays
words = np.empty(len(data), dtype=object)
labels = np.empty(len(data), dtype=object)

# Fill the arrays
for i, x in tqdm(enumerate(data), total=len(data)):
    words[i] = np.array(x["tokens"])
    labels[i] = np.array([CFG.label2id[label] for label in x["labels"]])

  0%|          | 0/6807 [00:00<?, ?it/s]

In [10]:
# Get unique labels and their frequency
all_labels = np.array([x for label in labels for x in label])
unique_labels, label_counts = np.unique(all_labels, return_counts=True)

# Plotting
fig = go.Figure(data=go.Bar(x=CFG.labels, y=label_counts))
fig.update_layout(
    title="Label Distribution",
    xaxis_title="Labels",
    yaxis_title="Count",
    yaxis_type="log",
)

fig.update_traces(text=label_counts, textposition="outside")
fig.show()

In [11]:
# Splitting the data into training and testing sets
train_words, valid_words, train_labels, valid_labels = train_test_split(
    words, labels, test_size=0.2, random_state=CFG.seed
)

In [12]:
# To convert string input or list of strings input to numerical tokens
tokenizer = keras_nlp.models.DebertaV3Tokenizer.from_preset(
    CFG.preset,
)

# Preprocessing layer to add spetical tokens: [CLS], [SEP], [PAD]
packer = keras_nlp.layers.MultiSegmentPacker(
    start_value=tokenizer.cls_token_id,
    end_value=tokenizer.sep_token_id,
    sequence_length=10,
)

Attaching 'tokenizer.json' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...


In [13]:
sample_words = words[0][:5]
sample_tokens_int = [
    token.tolist() for word in sample_words for token in tokenizer(word)
]
sample_tokens_str = [tokenizer.id_to_token(token) for token in sample_tokens_int]

print("words        :", sample_words.tolist())
print("tokens (str) :", sample_tokens_str)
print("tokens (int) :", sample_tokens_int)

words        : ['Design', 'Thinking', 'for', 'innovation', 'reflexion']
tokens (str) : ['▁Design', '▁Thinking', '▁for', '▁innovation', '▁reflex', 'ion']
tokens (int) : [2169, 12103, 270, 3513, 28310, 4593]


In [14]:
padded_sample_tokens_int = packer(np.array(sample_tokens_int))[0].tolist()
padded_sample_tokens_str = [
    tokenizer.id_to_token(token) for token in padded_sample_tokens_int
]

print("tokens (str)        :", sample_tokens_str)
print("padded tokens (str) :", padded_sample_tokens_str, "\n")

print("tokens (int)        :", sample_tokens_int)
print("padded tokens (int) :", padded_sample_tokens_int)

tokens (str)        : ['▁Design', '▁Thinking', '▁for', '▁innovation', '▁reflex', 'ion']
padded tokens (str) : ['[CLS]', '▁Design', '▁Thinking', '▁for', '▁innovation', '▁reflex', 'ion', '[SEP]', '[PAD]', '[PAD]'] 

tokens (int)        : [2169, 12103, 270, 3513, 28310, 4593]
padded tokens (int) : [1, 2169, 12103, 270, 3513, 28310, 4593, 2, 0, 0]


/opt/conda/lib/python3.10/site-packages/keras/src/backend/jax/core.py:64: UserWarning:

Explicitly requested dtype int64 requested in asarray is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.



In [15]:
def get_tokens(words, seq_len, packer):
    # Tokenize input
    token_words = tf.expand_dims(
        tokenizer(words), axis=-1
    )  # ex: (words) ["It's", "a", "cat"] ->  (token_words) [[1, 2], [3], [4]]
    tokens = tf.reshape(
        token_words, [-1]
    )  # ex: (token_words) [[1, 2], [3], [4]] -> (tokens) [1, 2, 3, 4]
    # Pad tokens
    tokens = packer(tokens)[0][:seq_len]
    inputs = {"token_ids": tokens, "padding_mask": tokens != 0}
    return inputs, tokens, token_words


def get_token_ids(token_words):
    # Get word indices
    word_ids = tf.range(tf.shape(token_words)[0])
    # Get size of each word
    word_size = tf.reshape(
        tf.map_fn(lambda word: tf.shape(word)[0:1], token_words), [-1]
    )
    # Repeat word_id with size of word to get token_id
    token_ids = tf.repeat(word_ids, word_size)
    return token_ids


def get_token_labels(word_labels, token_ids, seq_len):
    # Create token_labels from word_labels ->  alignment
    token_labels = tf.gather(word_labels, token_ids)
    # Only label the first token of a given word and assign -100 to others
    mask = tf.concat([[True], token_ids[1:] != token_ids[:-1]], axis=0)
    token_labels = tf.where(mask, token_labels, -100)
    # Truncate to max sequence length
    token_labels = token_labels[: seq_len - 2]  # -2 for special tokens ([CLS], [SEP])
    # Pad token_labels to align with tokens (use -100 to pad for loss/metric ignore)
    pad_start = 1  # for [CLS] token
    pad_end = seq_len - tf.shape(token_labels)[0] - 1  # for [SEP] and [PAD] tokens
    token_labels = tf.pad(token_labels, [[pad_start, pad_end]], constant_values=-100)
    return token_labels


def process_token_ids(token_ids, seq_len):
    # Truncate to max sequence length
    token_ids = token_ids[: seq_len - 2]  # -2 for special tokens ([CLS], [SEP])
    # Pad token_ids to align with tokens (use -1 to pad for later identification)
    pad_start = 1  # [CLS] token
    pad_end = seq_len - tf.shape(token_ids)[0] - 1  # [SEP] and [PAD] tokens
    token_ids = tf.pad(token_ids, [[pad_start, pad_end]], constant_values=-1)
    return token_ids


def process_data(seq_len=720, has_label=True, return_ids=False):
    # To add spetical tokens: [CLS], [SEP], [PAD]
    packer = keras_nlp.layers.MultiSegmentPacker(
        start_value=tokenizer.cls_token_id,
        end_value=tokenizer.sep_token_id,
        sequence_length=seq_len,
    )

    def process(x):
        # Generate inputs from tokens
        inputs, tokens, words_int = get_tokens(x["words"], seq_len, packer)
        # Generate token_ids for maping tokens to words
        token_ids = get_token_ids(words_int)
        if has_label:
            # Generate token_labels from word_labels
            token_labels = get_token_labels(x["labels"], token_ids, seq_len)
            return inputs, token_labels
        elif return_ids:
            # Pad token_ids to align with tokens
            token_ids = process_token_ids(token_ids, seq_len)
            return token_ids
        else:
            return inputs

    return process

In [16]:
def build_dataset(
    words,
    labels=None,
    return_ids=False,
    batch_size=4,
    seq_len=512,
    shuffle=False,
    cache=True,
    drop_remainder=True,
):
    AUTO = tf.data.AUTOTUNE

    slices = {"words": tf.ragged.constant(words)}
    if labels is not None:
        slices.update({"labels": tf.ragged.constant(labels)})

    ds = tf.data.Dataset.from_tensor_slices(slices)
    ds = ds.map(
        process_data(
            seq_len=seq_len, has_label=labels is not None, return_ids=return_ids
        ),
        num_parallel_calls=AUTO,
    )  # apply processing
    ds = ds.cache() if cache else ds  # cache dataset
    if shuffle:  # shuffle dataset
        ds = ds.shuffle(1024, seed=CFG.seed)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)
    ds = ds.batch(batch_size, drop_remainder=drop_remainder)  # batch dataset
    ds = ds.prefetch(AUTO)  # prefetch next batch
    return ds

In [17]:
train_ds = build_dataset(
    train_words,
    train_labels,
    batch_size=CFG.train_batch_size,
    seq_len=CFG.train_seq_len,
    shuffle=True,
)

valid_ds = build_dataset(
    valid_words,
    valid_labels,
    batch_size=CFG.train_batch_size,
    seq_len=CFG.train_seq_len,
    shuffle=False,
)

In [18]:
inp, tar = next(iter(valid_ds))
print("# Input:\n", inp)
print("\n# Labels:\n", tar)

# Input:
 {'token_ids': <tf.Tensor: shape=(16, 1024), dtype=int32, numpy=
array([[    1, 28525,   877, ...,     0,     0,     0],
       [    1, 45730,   377, ...,     0,     0,     0],
       [    1,  8489,  7933, ...,     0,     0,     0],
       ...,
       [    1,  6348, 28525, ...,  1068,  3955,     2],
       [    1,  3780, 13942, ...,     0,     0,     0],
       [    1, 45730,   377, ...,  6568,   366,     2]], dtype=int32)>, 'padding_mask': <tf.Tensor: shape=(16, 1024), dtype=bool, numpy=
array([[ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ...,  True,  True,  True]])>}

# Labels:
 tf.Tensor(
[[-100   12   12 ... -100 -100 -100]
 [-100   12   12 ... -100 -100 -100]
 [-100   12 -100 ... -100 -100 -100]
 ...
 [-100 

In [19]:
class CrossEntropy(keras.losses.SparseCategoricalCrossentropy):
    def __init__(self, ignore_class=-100, reduction=None, **args):
        super().__init__(reduction=reduction, **args)
        self.ignore_class = ignore_class

    def call(self, y_true, y_pred):
        y_true = ops.reshape(y_true, [-1])
        y_pred = ops.reshape(y_pred, [-1, CFG.num_labels])
        loss = super().call(y_true, y_pred)
        if self.ignore_class is not None:
            valid_mask = ops.not_equal(
                y_true, ops.cast(self.ignore_class, y_pred.dtype)
            )
            loss = ops.where(valid_mask, loss, 0.0)
            loss = ops.sum(loss)
            loss /= ops.maximum(ops.sum(ops.cast(valid_mask, loss.dtype)), 1)
        else:
            loss = ops.mean(loss)
        return loss

In [20]:
class FBetaScore(keras.metrics.FBetaScore):
    def __init__(
        self,
        ignore_classes=[-100, 12],
        average="micro",
        beta=5.0,
        name="f5_score",
        **args
    ):
        super().__init__(beta=beta, average=average, name=name, **args)
        self.ignore_classes = ignore_classes or []

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = ops.convert_to_tensor(y_true, dtype=self.dtype)
        y_pred = ops.convert_to_tensor(y_pred, dtype=self.dtype)

        y_true = ops.reshape(y_true, [-1])
        y_pred = ops.reshape(y_pred, [-1, CFG.num_labels])

        valid_mask = ops.ones_like(y_true, dtype=self.dtype)
        if self.ignore_classes:
            for ignore_class in self.ignore_classes:
                valid_mask &= ops.not_equal(
                    y_true, ops.cast(ignore_class, y_pred.dtype)
                )
        valid_mask = ops.expand_dims(valid_mask, axis=-1)

        y_true = ops.one_hot(y_true, CFG.num_labels)

        if not self._built:
            self._build(y_true.shape, y_pred.shape)

        threshold = ops.max(y_pred, axis=-1, keepdims=True)
        y_pred = ops.logical_and(y_pred >= threshold, ops.abs(y_pred) > 1e-9)

        y_pred = ops.cast(y_pred, dtype=self.dtype)
        y_true = ops.cast(y_true, dtype=self.dtype)

        tp = ops.sum(y_pred * y_true * valid_mask, self.axis)
        fp = ops.sum(y_pred * (1 - y_true) * valid_mask, self.axis)
        fn = ops.sum((1 - y_pred) * y_true * valid_mask, self.axis)

        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)

In [21]:
# Build Token Classification model
backbone = keras_nlp.models.DebertaV3Backbone.from_preset(
    CFG.preset,
)
out = backbone.output
out = keras.layers.Dense(CFG.num_labels, name="logits")(out)
out = keras.layers.Activation("softmax", dtype="float32", name="prediction")(out)
model = keras.models.Model(backbone.input, out)

# Compile model for optimizer, loss and metric
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=2e-5),
    loss=CrossEntropy(),
    metrics=[FBetaScore()],
)

# Print each layer's name and whether it is trainable
for layer in model.layers:
    print(layer.name, layer.trainable)

# Summary of the model architecture
model.summary()

Attaching 'config.json' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/deberta_v3/keras/deberta_v3_small_en/2' to your Kaggle notebook...


token_ids True
token_embedding True
embeddings_layer_norm True
embeddings_dropout True
padding_mask True
rel_embedding True
disentangled_attention_encoder_layer_0 True
disentangled_attention_encoder_layer_1 True
disentangled_attention_encoder_layer_2 True
disentangled_attention_encoder_layer_3 True
disentangled_attention_encoder_layer_4 True
disentangled_attention_encoder_layer_5 True
logits True
prediction True


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ token_ids           │ (None, None)      │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ token_embedding     │ (None, None, 768) │ 98,380… │ token_ids[0][0]      │
│ (ReversibleEmbeddi… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ embeddings_layer_n… │ (None, None, 768) │   1,536 │ token_embedding[0][… │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ embeddings_dropout  │ (None, None, 768) │       0 │ embeddings_layer_no… │
│ (Dropout)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ padding_mask        │ (None, None)      │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ rel_embedding       │ (None, 512, 768)  │ 394,752 │ embeddings_dropout[… │
│ (RelativeEmbedding) │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ disentangled_atten… │ (None, None, 768) │ 7,087,… │ embeddings_dropout[… │
│ (DisentangledAtten… │                   │         │ padding_mask[0][0],  │
│                     │                   │         │ rel_embedding[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ disentangled_atten… │ (None, None, 768) │ 7,087,… │ disentangled_attent… │
│ (DisentangledAtten… │                   │         │ padding_mask[0][0],  │
│                     │                   │         │ rel_embedding[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ disentangled_atten… │ (None, None, 768) │ 7,087,… │ disentangled_attent… │
│ (DisentangledAtten… │                   │         │ padding_mask[0][0],  │
│                     │                   │         │ rel_embedding[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ disentangled_atten… │ (None, None, 768) │ 7,087,… │ disentangled_attent… │
│ (DisentangledAtten… │                   │         │ padding_mask[0][0],  │
│                     │                   │         │ rel_embedding[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ disentangled_atten… │ (None, None, 768) │ 7,087,… │ disentangled_attent… │
│ (DisentangledAtten… │                   │         │ padding_mask[0][0],  │
│                     │                   │         │ rel_embedding[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ disentangled_atten… │ (None, None, 768) │ 7,087,… │ disentangled_attent… │
│ (DisentangledAtten… │                   │         │ padding_mask[0][0],  │
│                     │                   │         │ rel_embedding[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ logits (Dense)      │ (None, None, 13)  │   9,997 │ disentangled_attent… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ prediction          │ (None, None, 13)  │       0 │ logits[0][0]         │
│ (Activation)        │                   │         │                      │
└─────────────────────┴───────────────────┴─────────┴──────────────────────┘

 Total params: 141,314,317 (539.07 MB)

 Trainable params: 141,314,317 (539.07 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
import math


def get_lr_callback(batch_size=8, mode="cos", epochs=10, plot=False):
    lr_start, lr_max, lr_min = 6e-6, 2.5e-6 * batch_size, 1e-6
    lr_ramp_ep, lr_sus_ep, lr_decay = 3, 0, 0.75

    def lrfn(epoch):  # Learning rate update function
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
        elif mode == "exp":
            lr = (lr_max - lr_min) * lr_decay ** (
                epoch - lr_ramp_ep - lr_sus_ep
            ) + lr_min
        elif mode == "step":
            lr = lr_max * lr_decay ** ((epoch - lr_ramp_ep - lr_sus_ep) // 2)
        elif mode == "cos":
            decay_total_epochs, decay_epoch_index = (
                epochs - lr_ramp_ep - lr_sus_ep + 3,
                epoch - lr_ramp_ep - lr_sus_ep,
            )
            phase = math.pi * decay_epoch_index / decay_total_epochs
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        return lr

    if plot:  # Plot lr curve if plot is True
        fig = px.line(
            x=np.arange(epochs),
            y=[lrfn(epoch) for epoch in np.arange(epochs)],
            title="LR Scheduler",
            markers=True,
            labels={"x": "epoch", "y": "lr"},
        )
        fig.update_layout(yaxis=dict(showexponent="all", exponentformat="e"))
        fig.show()

    return keras.callbacks.LearningRateScheduler(
        lrfn, verbose=False
    )  # Create lr callback

In [23]:
lr_cb = get_lr_callback(CFG.train_batch_size, mode=CFG.lr_mode, plot=True)

In [24]:
if CFG.train:
    history = model.fit(
        train_ds,
        validation_data=valid_ds,
        epochs=CFG.epochs,
        callbacks=[lr_cb],
        verbose=1,
    )
else:
    model.load_weights("/kaggle/input/pii-data-detection-ckpt-ds/model.weights.h5")

Epoch 1/6
340/340 ━━━━━━━━━━━━━━━━━━━━ 633s 2s/step - f5_score: 0.0298 - loss: 0.4707 - val_f5_score: 0.0111 - val_loss: 0.0044 - learning_rate: 6.0000e-06
Epoch 2/6
340/340 ━━━━━━━━━━━━━━━━━━━━ 530s 2s/step - f5_score: 0.0993 - loss: 0.0032 - val_f5_score: 0.6107 - val_loss: 0.0013 - learning_rate: 1.7333e-05
Epoch 3/6
340/340 ━━━━━━━━━━━━━━━━━━━━ 526s 2s/step - f5_score: 0.5713 - loss: 0.0013 - val_f5_score: 0.8118 - val_loss: 6.7507e-04 - learning_rate: 2.8667e-05
Epoch 4/6
340/340 ━━━━━━━━━━━━━━━━━━━━ 518s 2s/step - f5_score: 0.7997 - loss: 5.9974e-04 - val_f5_score: 0.8838 - val_loss: 4.7267e-04 - learning_rate: 4.0000e-05
Epoch 5/6
340/340 ━━━━━━━━━━━━━━━━━━━━ 518s 2s/step - f5_score: 0.9213 - loss: 2.7220e-04 - val_f5_score: 0.8819 - val_loss: 4.3189e-04 - learning_rate: 3.0250e-05
Epoch 6/6
340/340 ━━━━━━━━━━━━━━━━━━━━ 514s 2s/step - f5_score: 0.9604 - loss: 1.4168e-04 - val_f5_score: 0.8745 - val_loss: 4.4768e-04 - learning_rate: 2.2938e-05


Even though this notebook does both training and inference, let's store the **weights** of the trained model on disk in case we may need it later. Also, if you need more time for inference, you can create separate notebooks for training and inference.

> **Note**: The filename of the weights should end in `.weights.h5`

In [25]:
model.save_weights("model.weights.h5")

In [26]:
# Build Validation dataloader with "infer_seq_len"
valid_ds = build_dataset(
    valid_words,
    valid_labels,
    return_ids=False,
    batch_size=CFG.infer_batch_size,
    seq_len=CFG.infer_seq_len,
    shuffle=False,
    cache=False,
)

In [27]:
# Evaluate
model.evaluate(valid_ds, return_dict=True, verbose=0)

{'f5_score': 0.8586761355400085, 'loss': 0.00047128699952736497}

In [28]:
# Test data
test_data = json.load(open(f"{BASE_PATH}/test.json"))

# Ensure number of samples is divisble by number of devices
need_samples = len(devices) - len(test_data) % len(devices)
for _ in range(need_samples):
    test_data.append(test_data[-1])  # repeat the last sample

# Initialize empty arrays
test_words = np.empty(len(test_data), dtype=object)
test_docs = np.empty(len(test_data), dtype=np.int32)

# Fill the arrays
for i, x in tqdm(enumerate(test_data), total=len(test_data)):
    test_words[i] = np.array(x["tokens"])
    test_docs[i] = x["document"]

# Get token ids
id_ds = build_dataset(
    test_words,
    return_ids=True,
    batch_size=len(test_words),
    seq_len=CFG.infer_seq_len,
    shuffle=False,
    cache=False,
    drop_remainder=False,
)
test_token_ids = ops.convert_to_numpy([ids for ids in iter(id_ds)][0])

# Build test dataloader
test_ds = build_dataset(
    test_words,
    return_ids=False,
    batch_size=CFG.infer_batch_size,
    seq_len=CFG.infer_seq_len,
    shuffle=False,
    cache=False,
    drop_remainder=False,
)

  0%|          | 0/12 [00:00<?, ?it/s]

In [29]:
# Do inference
test_preds = model.predict(test_ds, verbose=1)

# Convert probabilities to class labels via max confidence
test_preds = np.argmax(test_preds, axis=-1)

3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step


In [30]:
test_docs = test_docs[:-need_samples]
test_token_ids = test_token_ids[:-need_samples]
test_preds = test_preds[:-need_samples]
test_words = test_words[:-need_samples]

In [31]:
document_list = []
token_id_list = []
label_id_list = []
token_list = []

for doc, token_ids, preds, tokens in tqdm(
    zip(test_docs, test_token_ids, test_preds, test_words), total=len(test_words)
):
    # Create mask for filtering
    mask1 = np.concatenate(
        ([True], token_ids[1:] != token_ids[:-1])
    )  # ignore non-start tokens of a word
    mask2 = preds != 12  # ignore `O` (BIO format) label -> 12 (integer format) label
    mask3 = token_ids != -1  # ignore [CLS], [SEP], and [PAD] tokens
    mask = mask1 & mask2 & mask3  # merge filters

    # Apply filter
    token_ids = token_ids[mask]
    preds = preds[mask]

    # Store prediction if number of tokens is not zero
    if len(token_ids):
        token_list.extend(tokens[token_ids])
        document_list.extend([doc] * len(token_ids))
        token_id_list.extend(token_ids)
        label_id_list.extend(preds)

  0%|          | 0/10 [00:00<?, ?it/s]

In [32]:
pred_df = pd.DataFrame(
    {
        "document": document_list,
        "token": token_id_list,
        "label_id": label_id_list,
        "token_string": token_list,
    }
)
pred_df = pred_df.rename_axis("row_id").reset_index()  # add `row_id` column
pred_df["label"] = pred_df.label_id.map(
    CFG.id2label
)  # map integer label to BIO format label
pred_df.head(10)

,row_id,document,token,label_id,token_string,label
0,0,7,9,2,Nathalie,B-NAME_STUDENT
1,1,7,10,8,Sylla,I-NAME_STUDENT
2,2,7,482,2,Nathalie,B-NAME_STUDENT
3,3,7,483,8,Sylla,I-NAME_STUDENT
4,4,7,741,2,Nathalie,B-NAME_STUDENT
5,5,7,742,8,Sylla,I-NAME_STUDENT
6,6,10,0,2,Diego,B-NAME_STUDENT
7,7,10,1,8,Estrada,I-NAME_STUDENT
8,8,10,464,2,Diego,B-NAME_STUDENT
9,9,10,465,8,Estrada,I-NAME_STUDENT


In `submission.csv` we are excluding `token_string` and `label_id` from the columns as they are not part of submission file.

In [33]:
sub_df = pred_df.drop(columns=["token_string", "label_id"])  # remove extra columns
sub_df.to_csv("submission.csv", index=False)